# <p align="center">Siemens Sales Forecast</p>

---

## <p align="center">*2 - Feature Selection & Modeling*</p>

---

### 👥 **Team Members**
- **Ana Farinha** *(Student Number: 20211514)*  
- **António Oliveira** *(Student Number: 20211595)*  
- **Mariana Neto** *(Student Number: 20211527)*  
- **Salvador Domingues** *(Student Number: 20240597)*  

📅 **Date:** *April 1, 2025*  
📍 **Prepared for:** *Siemens*  

**GitHub Repo:** https://github.com/MGN19/Siemens-forecast

---

# ToC

<a class="anchor" id="top"></a>


1. [Import Libraries & Data](#1.-Import-Libraries-&-Data) <br><br>

2. [Product Category #1](#Product-Category-#1) <br><br>

In [1]:
## CELL TYPES (remover depois)

<div class="alert-danger">
    
test

<div class="alert-warning">
    
test

<div class="alert-info">
    
test

# 1. Import Libraries & Data

In [27]:
import os
import pandas as pd

pd.set_option('display.max_columns', None)

# Suppress Warnings
import warnings
warnings.filterwarnings("ignore")


**Data**

In [28]:
folder_path = './data/individual_dfs/'

# List to store the names of each dataframe
imported_df_names = []

for file in os.listdir(folder_path):
    if file.endswith('.csv'):
        df_name = file.replace('.csv', '')  # Get name without .csv
        df_path = os.path.join(folder_path, file)
        
        # Read CSV and assign to a variable
        globals()[df_name] = pd.read_csv(df_path, index_col=0)
        
        # Store the dataframe name in the list
        imported_df_names.append(df_name)

# Convert the index of all DataFrames to datetime
for df_name in imported_df_names:
    globals()[df_name].index = pd.to_datetime(globals()[df_name].index)

# Optional: print to check
print(imported_df_names)

['df_8_diff', 'df_14', 'df_16', 'df_12', 'df_13', 'df_9', 'df_11', 'df_8', 'df_5', 'df_20', 'df_4', 'df_36', 'df_6', 'df_3', 'df_1']


# Product Category #1

<a href="#top">Top &#129033;</a>

In [31]:
# pd.reset_option('display.max_rows', None)
# df_1.isna().sum()

In [32]:
df_1.head(3)

,#1,CHI Production Index,CHI Shipments Index,FRA Production Index,FRA Shipments Index,GER Production Index,GER Shipments Index,ITA Production Index,ITA Shipments Index,JAP Production Index,JAP Shipments Index,SWI Production Index,SWI Shipments Index,UK Production Index,UK Shipments Index,USA Production Index,USA Shipments Index,Europe Production Index,Europe Shipments Index,(W) Price of Base Metals,(W) Price of Energy,(W) Price of Metals & Minerals,(W) Price of Natural gas index,"(W) Price of Crude oil, average",(W) Price of Copper,USA EUR to LCU Conversion Rate,USA EE Producer Prices,UK EE Producer Prices,ITA EE Producer Prices,FRA EE Producer Prices,GER EE Producer Prices,CHI EE Producer Prices,USA Machinery & Equipment Index,(W) Machinery & Equipment Index,SWI Machinery & Equipment Index,UK Machinery & Equipment Index,ITA Machinery & Equipment Index,JAP Machinery & Equipment Index,FRA Machinery & Equipment Index,GER Machinery & Equipment Index,USA EE Production Index,(W) EE Production Index,SWI EE Production Index,UK EE Production Index,ITA EE Production Index,JAP EE Production Index,FRA EE Production Index,GER EE Production Index,CHI Production Index_Rolling_Mean_3,CHI Shipments Index_Rolling_Mean_3,FRA Production Index_Rolling_Mean_3,FRA Shipments Index_Rolling_Mean_3,GER Production Index_Rolling_Mean_3,GER Shipments Index_Rolling_Mean_3,ITA Production Index_Rolling_Mean_3,ITA Shipments Index_Rolling_Mean_3,JAP Production Index_Rolling_Mean_3,JAP Shipments Index_Rolling_Mean_3,SWI Production Index_Rolling_Mean_3,SWI Shipments Index_Rolling_Mean_3,UK Production Index_Rolling_Mean_3,UK Shipments Index_Rolling_Mean_3,USA Production Index_Rolling_Mean_3,USA Shipments Index_Rolling_Mean_3,Europe Production Index_Rolling_Mean_3,Europe Shipments Index_Rolling_Mean_3,(W) Price of Base Metals_Rolling_Mean_3,(W) Price of Energy_Rolling_Mean_3,(W) Price of Metals & Minerals_Rolling_Mean_3,(W) Price of Natural gas index_Rolling_Mean_3,"(W) Price of Crude oil, average_Rolling_Mean_3",(W) Price of Copper_Rolling_Mean_3,USA EUR to LCU Conversion Rate _Rolling_Mean_3,USA EE Producer Prices_Rolling_Mean_3,UK EE Producer Prices_Rolling_Mean_3,ITA EE Producer Prices_Rolling_Mean_3,FRA EE Producer Prices_Rolling_Mean_3,GER EE Producer Prices_Rolling_Mean_3,CHI EE Producer Prices_Rolling_Mean_3,USA Machinery & Equipment Index_Rolling_Mean_3,(W) Machinery & Equipment Index_Rolling_Mean_3,SWI Machinery & Equipment Index_Rolling_Mean_3,UK Machinery & Equipment Index_Rolling_Mean_3,ITA Machinery & Equipment Index_Rolling_Mean_3,JAP Machinery & Equipment Index_Rolling_Mean_3,FRA Machinery & Equipment Index_Rolling_Mean_3,GER Machinery & Equipment Index_Rolling_Mean_3,USA EE Production Index_Rolling_Mean_3,(W) EE Production Index_Rolling_Mean_3,SWI EE Production Index_Rolling_Mean_3,UK EE Production Index_Rolling_Mean_3,ITA EE Production Index_Rolling_Mean_3,JAP EE Production Index_Rolling_Mean_3,FRA EE Production Index_Rolling_Mean_3,GER EE Production Index_Rolling_Mean_3,CC_CHI,CC_FRA,CC_GER,CC_ITA,CC_JAP,CC_Europe,CC_SWI,CC_UK,CC_USA,BC_CHI,BC_FRA,BC_GER,BC_ITA,BC_JAP,BC_Europe,BC_SWI,BC_UK,BC_USA,stock_price,stock_price_change,stock_volume
2018-10-01,36098918.79,211.955755,211.955755,108.280608,122.451734,124.227879,137.741953,118.350514,122.456894,125.209957,124.793250,109.077781,104.594781,113.659322,112.318030,111.902540,127.808839,124.391967,130.989253,86.102586,100.222169,79.354986,89.570796,97.072264,82.545082,1.1484,110.700409,111.463669,105.297836,102.064743,109.119614,96.318329,111.422638,131.340118,106.816937,110.893450,129.389221,131.503786,114.720810,127.461136,112.853256,129.325775,112.970843,118.670791,93.001511,112.376774,97.849541,118.298233,202.967874,202.967874,91.380433,108.585140,119.151650,134.801449,94.859629,104.070613,122.745057,123.893424,106.056871,101.699118,108.542104,105.792730,111.222701,127.351648,114.021188,123.656133,85.406475,96.977598,78.306125,86.817312,94.117390,81.052581,1.156400,110.399208,113.306043,105.3

**Train-Validation Split**

In [33]:
def train_val_split(df, target_col, val_percentage):
    """
    Splits a time series dataframe into training and validation sets based on a percentage of validation data.
    
    Args:
        df (pd.DataFrame): The time series dataframe with a DateTime index.
        target_col (str): The name of the target column.
        val_percentage (float): The percentage of the data to use for validation (between 0 and 1).
        
    Returns:
        X_train, X_val, y_train, y_val (pd.DataFrames/Series)
    """
    n_obs = len(df)
    val_size = int(n_obs * val_percentage)
    split_index = n_obs - val_size
    
    train = df.iloc[:split_index]
    val = df.iloc[split_index:]
    
    X_train = train.drop(columns=[target_col])
    y_train = train[target_col]
    
    X_val = val.drop(columns=[target_col])
    y_val = val[target_col]
    
    return X_train, X_val, y_train, y_val


In [34]:
X_train_1, X_val_1, y_train_1, y_val_1 = train_val_split(df_1, '#1', val_percentage = 0.3)

**Scaling**

In [35]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

def scale_data(X_train, X_val, scaler_type='minmax'):
    """
    Scales the training and validation data using the specified scaler.
    
    Parameters:
        X_train (pd.DataFrame): The training features.
        X_val (pd.DataFrame): The validation features.
        scaler_type (str): The type of scaler to use.
    
    Returns:
        X_train_scaled (pd.DataFrame): Scaled training features.
        X_val_scaled (pd.DataFrame): Scaled validation features.
    """
    # Choose scaler based on input
    if scaler_type == 'minmax':
        scaler = MinMaxScaler()
    elif scaler_type == 'standard':
        scaler = StandardScaler()
    elif scaler_type == 'robust':
        scaler = RobustScaler()
    else:
        raise ValueError("Invalid scaler type. Choose either 'minmax' or 'standard'.")
    
    # Fit on training data and transform it
    X_train_scaled = scaler.fit_transform(X_train)
    
    # Transform the validation data using the same scaler
    X_val_scaled = scaler.transform(X_val)
    
    # Convert back to DataFrame to keep column names
    X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
    X_val_scaled = pd.DataFrame(X_val_scaled, columns=X_val.columns, index=X_val.index)
    
    return X_train_scaled, X_val_scaled


In [36]:
X_train_1_scaled, X_val_1_scaled = scale_data(X_train_1, 
                                          X_val_1, 
                                          scaler_type='minmax')

## 2.1 Feature Selection

In [54]:
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVR

def feature_selection(X_train, y_train, method='all', 
                      k=10, rfe_model=None):
    
    selected_features = []
    
    if method == 'correlation':
        # Correlation-based Feature Selection
        corr_matrix = X_train.corr()
        upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        correlated_features = [column for column in upper_triangle.columns if any(upper_triangle[column].abs() > 0.9)]
        selected_features = [col for col in X_train.columns if col not in correlated_features]
        print(f'Selected {len(selected_features)} features ')
    
    # Univariate Feature Selection
    elif method == 'univariate':
        selector = SelectKBest(score_func=f_classif, k=k)
        selector.fit(X_train, y_train)
        selected_features = X_train.columns[selector.get_support()]
        print(f'Selected {len(selected_features)} features')
    
    # RFE
    elif method == 'rfe':
        if rfe_model is None:
            # Default to Linear Regression if no model is provided
            rfe_model = LinearRegression()
        
        rfe = RFE(rfe_model, n_features_to_select=k)
        rfe.fit(X_train, y_train)
        selected_features = X_train.columns[rfe.support_]
        print(f'Selected {len(selected_features)} features')
    
    # Feature Importance
    elif method == 'importance':
        model = RandomForestRegressor()
        model.fit(X_train, y_train)
        importance = model.feature_importances_
        feature_importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': importance})
        feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
        selected_features = feature_importance_df.head(k)['Feature'].tolist()
        print(f'Selected {len(selected_features)} features')
    
    # All Methods
    elif method == 'all':
        corr_features = feature_selection(X_train, y_train, method='correlation', k=k, rfe_model=rfe_model)
        univariate_features = feature_selection(X_train, y_train, method='univariate', k=k, rfe_model=rfe_model)
        rfe_features = feature_selection(X_train, y_train, method='rfe', k=k, rfe_model=rfe_model)
        importance_features = feature_selection(X_train, y_train, method='importance', k=k, rfe_model=rfe_model)
        
        # Intersect the selected features from all methods
        selected_features = list(set(corr_features) & set(univariate_features) & set(rfe_features) & set(importance_features))
    
    return selected_features

In [56]:
selected_features = feature_selection(X_train_1_scaled, y_train_1, 
                  method='all', 
                  k=50)

selected_features

Selected 36 features 
Selected 50 features
Selected 50 features
Selected 50 features


['ITA Shipments Index',
 'ITA EE Producer Prices',
 'CHI Production Index',
 'USA Shipments Index_Rolling_Mean_3',
 'UK EE Production Index']

In [67]:
import lazypredict
from lazypredict.Supervised import LazyRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

def lazy_regressor(X_train, y_train, X_test, y_test, metric='RMSE'):
    
    # Initialize LazyRegressor
    regressor = LazyRegressor(verbose=0)
    
    # Fit the model
    models, predictions = regressor.fit(X_train, X_test, y_train, y_test)
    
    # Select the best model based on the metric (e.g., RMSE)
    best_model = models.sort_values(by=metric).iloc[0]
    
    # Get the model name and the best score
    best_model_name = best_model.name
    best_score = best_model[metric]
    
    print(f'Best model: {best_model_name}')
    print(f'{metric} of the best model: {best_score}')
    
    # Return the best model and score
    return best_model_name, best_score


In [68]:
best_model_name, best_score = lazy_regressor(X_train_1_scaled, y_train_1,
                                             X_val_1_scaled, y_val_1,
                                             metric='RMSE')


100%|████████████████████████████████████| 42/42 [00:02<00:00, 17.66it/s]

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 31, number of used features: 0
[LightGBM] [Info] Start training from score 36289610.258065
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet t

In [75]:
from autosklearn.experimental.askl2 import ForecasterAutoregMultiSeries
from sklearn.metrics import mean_squared_error
import numpy as np

def time_series_forecasting(X_train, y_train, X_test, y_test, 
                            forecast_steps=10, metric='neg_root_mean_squared_error'):

    # Initialize ForecasterAutoregMultiSeries
    forecaster = ForecasterAutoregMultiSeries(steps=forecast_steps, metric=metric)
    
    # Fit the forecaster with training data
    forecaster.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred = forecaster.predict(X_test)
    
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print(f'Test RMSE: {rmse}')
    
    return forecaster, y_pred, rmse


ModuleNotFoundError: No module named 'autosklearn'

### 2.1.1 Filter Methods

**Variance Threshold**

In [ ]:
X_train_1_scaled.var() 

**Spearman Correlation**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def correlation_matrix(X, cmap='YlOrBr', font_size = 5):
    """
    Input: X (numerical data)
    Output: Correlation Matrix
    """
    
    # Correlation matrix
    corr_matrix = X.corr().abs()

    # Plot Heatmap
    plt.figure(figsize=(12, 10))
    sns.heatmap(corr_matrix, annot=True, cmap=cmap, fmt=".2f", annot_kws={"size": font_size})
    plt.title("Feature Correlation Matrix")
    plt.show()

In [ ]:
correlation_matrix(X_train_1_scaled)

In [ ]:
def top_correlated_features(X, top_n=20, cmap='YlOrBr'):
    corr_matrix = X.corr().abs()
    mean_corr = corr_matrix.mean().sort_values(ascending=False)
    top_features = mean_corr.head(top_n).index
    plt.figure(figsize=(10, 8))
    sns.heatmap(X[top_features].corr(), annot=True, cmap=cmap, fmt=".2f", annot_kws={"size": 8})
    plt.title(f"Top {top_n} Most Correlated Features")
    plt.show()


In [ ]:
top_correlated_features(X_train_1_scaled)

### 2.2.2 Wrapper Methods

**RFE**

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

def rfe(X_train, y_train, X_val, y_val, n_features, model=None):
    """
    Input: 
        X_train, y_train, X_val, y_val: training and validation data
        n_features: number of features to use for RFE
        model: chosen regression model 
    Output: selected features for the best model based on RMSE
    """
    
    best_rmse = np.inf  # Start with a very high RMSE (lower is better)
    best_features = []

    results = {}
    
    for feature in n_features:
        
        # Fit RFE
        rfe = RFE(estimator=model, n_features_to_select=feature)
        rfe.fit(X_train, y_train)

        # Get selected features
        selected_features = X_train.columns[rfe.support_]
        
        print('\n-------------TRAIN-------------')

        # Predictions for Train
        y_pred_train = rfe.predict(X_train)
        
        # Metrics for Training (RMSE)
        mse_train = mean_squared_error(y_train, y_pred_train)
        rmse_train = np.sqrt(mse_train)
        
        print(f"RMSE for {feature} features (Train): {rmse_train:.4f}")
        
        print('----------VALIDATION----------')

        # Predictions for Validation
        y_pred_val = rfe.predict(X_val)
        
        # Metrics for Validation (RMSE)
        mse_val = mean_squared_error(y_val, y_pred_val)
        rmse_val = np.sqrt(mse_val)
        
        print(f"RMSE for {feature} features (Validation): {rmse_val:.4f}")
        
        # Best score based on RMSE (lower is better)
        if rmse_val < best_rmse:
            best_rmse = rmse_val
            best_features = selected_features.tolist()  
        
    print('\n----------FINAL----------')
    print(f'Best RMSE: {best_rmse}')
    print(f'Number of Features: {len(best_features)}')
    print(f'Features: {best_features}')
    


In [ ]:
## TEMP
X_train_1_scaled.fillna(0, inplace = True)
X_val_1_scaled.fillna(0, inplace = True)

n_features = np.arange(1, len(X_train_1_scaled.columns) + 1)
model = LinearRegression()

rfe(X_train_1_scaled, y_train_1, X_val_1_scaled, y_val_1, 
                                n_features = n_features, 
                                model = model)

In [ ]:
## TEMP
X_train_1_scaled.fillna(0, inplace = True)
X_val_1_scaled.fillna(0, inplace = True)


from xgboost import XGBRegressor

n_features = np.arange(1, len(X_train_1_scaled.columns) + 1)
model = XGBRegressor(objective='reg:squarederror',
                     random_state = 100)

rfe(X_train_1_scaled, y_train_1, X_val_1_scaled, y_val_1, 
                                n_features = n_features, 
                                model = model)

### 2.2.3 Embedded Methods

**LASSO**

In [ ]:
from sklearn.linear_model import Lasso
import functions as f
def lasso(X, y, alpha = 0.01, color = f.main_color):

    """
    Input: 
        X, y: data
        alpha: parameter for lasso
    Output: Plot, initial and selected features
    """
    
    # Fit
    lasso = Lasso(alpha=alpha)
    lasso.fit(X, y)
    
    # Get Feature Importance
    importance = pd.Series(lasso.coef_, index=X.columns)
    importance.sort_values().plot(kind="barh", color=color)
    non_zero_importance = importance[importance != 0]
    selected_features = non_zero_importance.index

    # Plot
    plt.title("Lasso Feature Importance")
    plt.xlabel("Coefficient Value")
    plt.show()
    
    # Print Results
    print(f"\nInitial Features: {len(X.columns)}\n")
    print(X.columns.tolist())
    print(f"\nDecision for Numerical Features (lasso ≠ 0): {len(selected_features.tolist())}\n")
    print(selected_features.tolist())


In [ ]:
lasso(X_train_1_scaled, y_train_1, alpha = 0.05)

# Model

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import numpy as np

# Initialize the XGBoost model
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)

# Fit the model using the training data
model.fit(X_train_1_scaled, y_train_1)

# Make predictions on the validation set
y_pred_val_1 = model.predict(X_val_1_scaled)

# Calculate RMSE for the validation set
rmse_val_1 = np.sqrt(mean_squared_error(y_val_1, y_pred_val_1))
print(f'Root Mean Squared Error on Validation Set: {rmse_val_1}')

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

plt.plot(y_val_1.index, y_val_1, label='Actual Validation Values', linestyle='-', color='g')
plt.plot(y_val_1.index, y_pred_val_1, label='Predicted Validation Values', linestyle='--', color='g')

plt.plot(y_train_1.index, y_train_1, label='Actual Training Values', linestyle='-', color='b')

plt.xlabel('Date/Time')
plt.ylabel('Sales')
plt.title('XGBoost Forecasting: Actual vs Predicted (Training and Validation)')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
